# Creating a Clients/Comapnies Dataset
A client requested for the creation of a dataset of their potential customers. These customers are institutions that requires their services. Prior to this request, the client rely on dataof potential clients who have made contact enquiries about the services provided by the client including those made via emails, phone calls, the business website, social media and other marketing channels. 

Although specific clients can be identified through other channels like looking into the correlation between clients, sector, location, to mention a few, datasets acquired by the clients business if successful, would have some advantage over the other channels because it can be applied to target customers who could be missed by the other marketing channels. Creating this kind of dataset was quiet challenging. I demonstrate in this notebook how I carried out the task. Here, I used the UK Government's Companies House [database](https://data.gov.uk/dataset/4462e41a-7413-4359-97f5-420b6ca5f9c0/basic-company-data) at data.gov.uk. This database is selected for this project because it contains data of the parent companies of every business in the UK hence deem perfect for demonstrating of how the task could be carried out to create the clients database.   

### Required Packages

- `IDE or Jupyter notebook` 

- `Pandas`

- `basename` from `Os.path` for retrieving data using path names, merging, normalizing

- `Urllib.request`to fetch web data

- `Zipfile` to read, write, append and create ZIP files

- `Glob` to retrieve files from directory or pathnames matching a specified pattern.

We beging by loading the pakages.

In [1]:
# load packages
from os.path import basename
import urllib.request
import requests
import zipfile
import glob
import pandas as pd

### Download Data
We download the companies data from UK Government's Companies House [database](https://data.gov.uk/dataset/4462e41a-7413-4359-97f5-420b6ca5f9c0/basic-company-data). This basic companies data was last updated on the September 27, 2019 and is available as six separate zip files. We create a list of the URLs of the zipfiles to download the data.

In [2]:
# create a list of the data urls
urls =["http://download.companieshouse.gov.uk/BasicCompanyData-part1.zip",
      "http://download.companieshouse.gov.uk/BasicCompanyData-part2.zip",
      "http://download.companieshouse.gov.uk/BasicCompanyData-part3.zip",
      "http://download.companieshouse.gov.uk/BasicCompanyData-part4.zip",
      "http://download.companieshouse.gov.uk/BasicCompanyData-part5.zip",
      "http://download.companieshouse.gov.uk/BasicCompanyData-part6.zip"]

We use a conditional "for loop" statement to loop through each of the URLs in the list of URLs to 

- print Downloading and the url to indicate the being downloaded,

- create a HTTP request by passing `url` to `urllib.request.urlopen()` function and store as the response to the request, 

- pass the `response.url` to `basename()` function to obtain the filename component, then 

- pass the `url` and `filename` to the `urllib.request.urlretrieve()` function to download each of the files into a target directory.

In [3]:
for url in urls:
    print('Downloading:', url)
    response = requests.get(url)
    filename = basename(response.url) 
    urllib.request.urlretrieve(url,filename)

Downloading: http://download.companieshouse.gov.uk/BasicCompanyData-part1.zip
Downloading: http://download.companieshouse.gov.uk/BasicCompanyData-part2.zip
Downloading: http://download.companieshouse.gov.uk/BasicCompanyData-part3.zip
Downloading: http://download.companieshouse.gov.uk/BasicCompanyData-part4.zip
Downloading: http://download.companieshouse.gov.uk/BasicCompanyData-part5.zip
Downloading: http://download.companieshouse.gov.uk/BasicCompanyData-part6.zip


### Unzip the Files
We pass the zip files downloaded into the working directory (`*.zip`) to the `glob.glob()` function to search the downloaded files. 

In [4]:
# search for downloaded zip files 
files = glob.glob('*.zip')
# list the files
files

['BasicCompanyData-part1.zip',
 'BasicCompanyData-part2.zip',
 'BasicCompanyData-part3.zip',
 'BasicCompanyData-part4.zip',
 'BasicCompanyData-part5.zip',
 'BasicCompanyData-part6.zip']

We apply a conditional statement (`for loop`) to loop through the files, pass `file` and the regular expression for 'raw string' (`'r'`) to the `zipfile.Zipfile()` function to unzip the files, then extract the contents with `unzip.extractall()` and save to a new directory.

In [5]:
for file in files:
    print('Unzipping:',file)
    
    with zipfile.ZipFile(file, 'r') as unzip:
        unzip.extractall('data/raw')

Unzipping: BasicCompanyData-part1.zip
Unzipping: BasicCompanyData-part2.zip
Unzipping: BasicCompanyData-part3.zip
Unzipping: BasicCompanyData-part4.zip
Unzipping: BasicCompanyData-part5.zip
Unzipping: BasicCompanyData-part6.zip


### Merge Files
We pass the list of CSV files in the new directory to the `glob.glob()` function search and list all the CSV files in the directory and assign their file paths.

In [6]:
# search for csv files in the directory
csv_files = glob.glob('data/raw/*.csv')
# list files
csv_files

['data/raw\\BasicCompanyData-2023-02-01-part1_7.csv',
 'data/raw\\BasicCompanyData-2023-02-01-part2_7.csv',
 'data/raw\\BasicCompanyData-2023-02-01-part3_7.csv',
 'data/raw\\BasicCompanyData-2023-02-01-part4_7.csv',
 'data/raw\\BasicCompanyData-2023-02-01-part5_7.csv',
 'data/raw\\BasicCompanyData-2023-02-01-part6_7.csv']

We create a new empty list for all_files, use conditional "for loop" statement to loop each filename in the `csv_files`, pass the filename to `pd.read_csv()`, and pass the dataframe to `.append()` function to add all the files together. 

In [7]:
all_files = []

for filename in csv_files:
    df = pd.read_csv(filename, index_col=None, header=0, low_memory=False)
    all_files.append(df)

Pass the list of files (`all_files`) to the `pd.concat()` function to obtaining the strings of data from all 6 CSV files together into a a single Pandas dataframe.

In [8]:
# combine the list of data into one df
df_companies = pd.concat(all_files, axis=0, ignore_index=True)

We inspect the first five rows of the data.

In [9]:
df_companies.head().T

,0,1,2,3,4
CompanyName,! HEAL UR TECH LTD,! LTD,!? LTD,!BIG IMPACT GRAPHICS LIMITED,!GOBERUB LTD
CompanyNumber,14413082,08209948,11399177,11743365,13404790
RegAddress.CareOf,NaN,NaN,NaN,NaN,NaN
RegAddress.POBox,NaN,NaN,NaN,NaN,NaN
RegAddress.AddressLine1,5 BRIDGE STREET,METROHOUSE 57 PEPPER ROAD,THE STUDIO HATHERLOW HOUSE,372 OLD STREET,13 SOUTHMILL ROAD
RegAddress.AddressLine2,NaN,HUNSLET,HATHERLOW,335 ROSDEN HOUSE,NaN
RegAddress.PostTown,GUILDFORD,LEEDS,ROMILEY,LONDON,BISHOP'S STORTFORD
RegAddress.County,NaN,YORKSHIRE,NaN,NaN,HERTFORDSHIRE
RegAddress.Country,ENGLAND,NaN,UNITED KINGDOM,UNITED KINGDOM,ENGLAND
RegAddress.PostCode,GU1 4RY,LS10 2RU,SK6 3DY,EC1V 9LT,CM23 3FR


We inspect the shape of the dataset and observe that it contains the list of about 5.1 million registered companies. This is the list of the head office of the business registered with Companies House. 

In [10]:
# get data info
df_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5099999 entries, 0 to 5099998
Data columns (total 55 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   CompanyName                         object 
 1    CompanyNumber                      object 
 2   RegAddress.CareOf                   object 
 3   RegAddress.POBox                    object 
 4   RegAddress.AddressLine1             object 
 5    RegAddress.AddressLine2            object 
 6   RegAddress.PostTown                 object 
 7   RegAddress.County                   object 
 8   RegAddress.Country                  object 
 9   RegAddress.PostCode                 object 
 10  CompanyCategory                     object 
 11  CompanyStatus                       object 
 12  CountryOfOrigin                     object 
 13  DissolutionDate                     float64
 14  IncorporationDate                   object 
 15  Accounts.AccountRefDay              float64
 16  

We notice that the dataframe obtain consist of the records of 5,099,999 companies with 55 fields. The fields includes Company Name, Registered Address, Company Incorporation Date and Status, Registered Partnerships/Partners. We proceed to write the data into a CSV file for the client company for their marketing campains.

In [11]:
# write df to csv 
df_companies.to_csv('companies.csv', index=False)

**George Jordan, May 7th 2022.**